In [1]:
import importlib
import numpy as np
from numba import njit
from functools import partial
import talib 
import pandas as pd
import vectorbt as vbt
import vectorbt as vbt
from vectorbt.utils.colors import adjust_opacity
from vectorbt.utils.enum_ import map_enum_fields
from vectorbt.base.reshape_fns import broadcast, flex_select_auto_nb, to_2d_array
from vectorbt.portfolio.enums import SizeType, Direction, NoOrder, OrderStatus, OrderSide
from vectorbt.portfolio import nb
import sys
import os
function_essential_path = r"H:\Desktop\Environement_Trading_Developement\FunctionEssential"
sys.path.append(function_essential_path)
import utils as us
importlib.reload(us)

<module 'utils' from 'H:\\Desktop\\Environement_Trading_Developement\\FunctionEssential\\utils.py'>

In [2]:
##FONCTION##


def tdi_maker(close,rsi_period,TDI_fast_period,TDI_low_period,TDI_midle_period):
    rsi = vbt.RSI.run(close,window=rsi_period).rsi
    fast_ma = vbt.MA.run(rsi, window=TDI_fast_period).ma
    low_ma = vbt.MA.run(rsi, window=TDI_low_period).ma
    midle_ma = vbt.MA.run(rsi, window=TDI_midle_period).ma
    tdi = pd.DataFrame({    
        "fast":fast_ma,
        "low":low_ma,
        "midle":midle_ma
    })
    return tdi

#Cette fonction marque Bull/Bear quand y'a un cross et Nothing quand y'a rien et dans le TDI et sur la seconde collone y'a l'angle du cross 
def tdi_cross_angle(tdi,rsi_period):
    cross = np.full(len(tdi), "Nothing", dtype=object)
    trend = np.full(len(tdi), "Nothing", dtype=object)  
    angle = np.zeros(len(tdi), dtype=float)
    fast_shifted = tdi["fast"].shift(1)  
    low_shifted = tdi["low"].shift(1)
    midle_shifted = tdi["midle"].shift(1)

    for idx in range(2,len(tdi)):
        if np.isnan(fast_shifted.iloc[idx]) or np.isnan(low_shifted.iloc[idx]):
            angle[idx] = np.nan
        else:
            angle_fast = np.arctan(tdi['fast'].iloc[idx] - fast_shifted.iloc[idx]) * 180 / np.pi
            angle_low = np.arctan(tdi['low'].iloc[idx] - low_shifted.iloc[idx]) * 180 / np.pi
            angle[idx] = (angle_low + (angle_fast * rsi_period / tdi['midle'].iloc[idx]))/(1+rsi_period / tdi['midle'].iloc[idx])
            trend[idx] = 'Bull' if midle_shifted.iloc[idx] < tdi['midle'].iloc[idx] else \
             'Bear' if midle_shifted.iloc[idx] > tdi['midle'].iloc[idx] else "Nothing"

        if tdi["fast"].iloc[idx-2] < tdi["low"].iloc[idx-2]:
            if tdi["fast"].iloc[idx] > tdi["low"].iloc[idx]:
                cross[idx] = "Bull"

        if tdi["fast"].iloc[idx-2] > tdi["low"].iloc[idx-2]:
            if tdi["fast"].iloc[idx] < tdi["low"].iloc[idx]:
                cross[idx] = "Bear"

    cross_angle_trend = pd.DataFrame({
        "cross":cross,
        "trend": trend,
        "angle":angle
    },index=tdi.index)

    return cross_angle_trend


def pivot_points(high, low, close):
    pivot = (high + low + close) / 3
    r1 = (2 * pivot) - low
    s1 = (2 * pivot) - high
    r2 = pivot + (high - low)
    s2 = pivot - (high - low)
    r3 = pivot + 2 * (high - low)  
    s3 = pivot - 2 * (high - low)  
    return r1, r2, r3, s1, s2, s3


In [9]:
##FONCTION LOGIQUE SIGNALS##



def entries_trades(close_m15, cross_m15, cross_m30, cross_h1, cross_h4, cross_d1):
    long = np.full_like(cross_m15, False, dtype=bool)
    short = np.full_like(cross_m15, False, dtype=bool)

    for idx in range(len(close_m15)):


        if (cross_m15[idx, 0] == 'Bull' and cross_m30[idx, 0] == 'Bull' and 
            cross_h1[idx, 0] == 'Bull' and cross_h4[idx, 0] == 'Bull' and 
            cross_d1[idx, 0] == 'Bull'):
            
            if (20 <= cross_m15[idx, 2] <= 80 and 20 <= cross_m30[idx, 2] <= 80 and 
                20 <= cross_h1[idx, 2] <= 80 and 20 <= cross_h4[idx, 2] <= 80 and 
                20 <= cross_d1[idx, 2] <= 80):
                    if (cross_m15[idx, 1] == 'Bull' and cross_m30[idx, 1] == 'Bull' and 
                    cross_h1[idx, 1] == 'Bull' and cross_h4[idx, 1] == 'Bull' and 
                    cross_d1[idx, 1] == 'Bull'):
                        long[idx, 0] = True

        # Vérifier conditions pour SHORT
        if (cross_m15[idx, 0] == 'Bear' and cross_m30[idx, 0] == 'Bear' and 
            cross_h1[idx, 0] == 'Bear' and cross_h4[idx, 0] == 'Bear' and 
            cross_d1[idx, 0] == 'Bear'):
            
            if (-20 >= cross_m15[idx, 2] >= -80 and -20 >= cross_m30[idx, 2] >= -80 and 
                -20 >= cross_h1[idx, 2] >= -80 and -20 >= cross_h4[idx, 2] >= -80 and 
                -20 >= cross_d1[idx, 2] >= -80):
                    if (cross_m15[idx, 1] == 'Bear' and cross_m30[idx, 1] == 'Bear' and 
                    cross_h1[idx, 1] == 'Bear' and cross_h4[idx, 1] == 'Bear' and 
                    cross_d1[idx, 1] == 'Bear'):

                        short[idx, 0] = True

    return long, short

    


def exits_trades(long,short,close,high,low,r3,s3):
    long_exit = np.zeros_like(long, dtype=np.float64)
    short_exit = np.zeros_like(short, dtype=np.float64)

    for col in range(long.shape[1]):
        for idx in range(len(close)-15):    

            if long[idx, col]:
                for j in range(idx+1,len(close)):
                    if low[j, col] <= s3[j, col]:
                        long_exit[j, col] = s3[j, col]
                        break
                    if high[j, col] >= r3[j, col]:
                        long_exit[j, col] = r3[j, col]
                        break
            
            if short[idx, col]:
                for j in range(idx+1,len(close)):
                    if high[j, col] >= r3[j, col]:
                        short_exit[j, col] = r3[j, col]
                        break
                    if low[j, col] <= s3[j, col]:
                        short_exit[j, col] = s3[j, col]
                        break
    
    return long_exit,short_exit

In [10]:
##INDICATEUR FACTORY##

Entry = vbt.IndicatorFactory(
    class_name="entries_trades",
    input_names=["close_m15","cross_m15","cross_m30","cross_h1","cross_h4","cross_d1"],
    output_names=["long","short"],
).from_apply_func(entries_trades)

Exits = vbt.IndicatorFactory(
    class_name="exits_trades",
    input_names=["long","short","close","high","low","r3","s3"],
    output_names=["long","short"],
).from_apply_func(exits_trades)



In [11]:
@njit
def pre_sim_func_nb(c):
    entry_price = np.full(c.target_shape[1], np.nan, dtype=np.float64)  
    return (entry_price,)

@njit
def order_func_nb(c, entry_price, long, short, long_exit,short_exit, size):
    price_now = nb.get_elem_nb(c, c.close)
    entry_long_now = nb.get_elem_nb(c, long)
    entry_short_now = nb.get_elem_nb(c, short)


    if entry_long_now:
        if c.position_now == 0:
            return nb.order_nb(
                size[c.col], 
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.001
            )
        elif c.position_now < 0:
            return nb.order_nb(
                -size[c.col],  
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.0005
            )  

    if entry_short_now:
        if c.position_now == 0:
            return nb.order_nb(
                size[c.col], 
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.001
            )
        elif c.position_now > 0:
            return nb.order_nb(
                -size[c.col],  
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.0005
            )

    #Si la valeur de l'idx de la sortie est différente de 0 ca veut dire que on doit out et on récupere la valeur de notre np.array long_exit 
    if c.position_now > 0:
        long_exit_now = nb.get_elem_nb(c, long_exit)
        if long_exit_now != 0:
            return nb.order_nb(
                -size[c.col],
                price=long_exit_now,
                direction=Direction.LongOnly,
                fees=0.0005
            )

    if c.position_now < 0:
        short_exit_now = nb.get_elem_nb(c, short_exit)
        if short_exit_now != 0:
            return nb.order_nb(
                -size[c.col],
                price=short_exit_now,
                direction=Direction.ShortOnly,
                fees=0.0005
            )
       

    return NoOrder

@njit
def post_order_func_nb(c, entry_price):
    if c.order_result.status == OrderStatus.Filled:
        entry_price[c.col] = c.order_result.price

In [21]:
##CLASS STRATEGIE##

class Strategie():

    params_fixe_un = "" #Peut etre un stop ou un TP ou autre
    params_fixe_deux = ""


    def __init__(self,data,tickers,frequence):
        self.data = data
        self.tickers = tickers
        self.frequences = frequence


    def set_params_strategie(self,main_frequence,rsi_period,tdi_fast_period,tdi_low_period,tdi_midle_period):
        self.main_frequence   = main_frequence
        self.rsi_period       = rsi_period
        self.tdi_fast_period  = tdi_fast_period
        self.tdi_low_period   = tdi_low_period
        self.tdi_midle_period = tdi_midle_period



    def backtest(self):
        entries_long_df = pd.DataFrame()
        entries_short_df = pd.DataFrame()
        exits_long_df = pd.DataFrame()
        exits_short_df = pd.DataFrame()
        close_df = pd.DataFrame()
        datetime_df = pd.DataFrame()

        for ticker in self.tickers:
            data = self.data[self.main_frequence][ticker]
            #Creation des entréer / Sortie / Indicateur

            close_m15 =   self.data['M15'][ticker]['close']
            high_m15 =   self.data['M15'][ticker]['high']
            low_m15 =   self.data['M15'][ticker]['low']
            datetime = data.reset_index()['Datetime']

            close_m30 =   self.data['M30'][ticker]['close']
            close_h1  =   self.data['H1'][ticker]['close']
            close_h4  =   self.data['H4'][ticker]['close']
            close_d1  =   self.data['D1'][ticker]['close']

            close_w1  =   self.data['W1'][ticker]['close']
            high_w1   =   self.data['W1'][ticker]['high']
            low_w1    =   self.data['W1'][ticker]['low']

            _,_,r3,_,_,s3 = pivot_points(high_w1, low_w1, close_w1)

            r3_ajuster = us.resample_shape_frequence(close_m15,r3)
            s3_ajuster = us.resample_shape_frequence(close_m15,s3)

            tdi_m15 = tdi_maker(close_m15,self.rsi_period,self.tdi_fast_period,self.tdi_low_period,self.tdi_midle_period)
            tdi_m30 = tdi_maker(close_m30,self.rsi_period,self.tdi_fast_period,self.tdi_low_period,self.tdi_midle_period)
            tdi_h1  = tdi_maker(close_h1,self.rsi_period,self.tdi_fast_period,self.tdi_low_period,self.tdi_midle_period)
            tdi_h4  = tdi_maker(close_h4,self.rsi_period,self.tdi_fast_period,self.tdi_low_period,self.tdi_midle_period)
            tdi_d1  = tdi_maker(close_d1,self.rsi_period,self.tdi_fast_period,self.tdi_low_period,self.tdi_midle_period)

            cross_angle_m15 = tdi_cross_angle(tdi_m15,self.rsi_period)
            cross_angle_m30 = tdi_cross_angle(tdi_m30,self.rsi_period)
            cross_angle_h1 = tdi_cross_angle(tdi_h1,self.rsi_period)
            cross_angle_h4 = tdi_cross_angle(tdi_h4,self.rsi_period)
            cross_angle_d1 = tdi_cross_angle(tdi_d1,self.rsi_period)

            cross_angle_m15_ajusted = us.resample_shape_frequence(close_m15,cross_angle_m15)
            cross_angle_m30_ajusted = us.resample_shape_frequence(close_m15,cross_angle_m30)
            cross_angle_h1_ajusted  = us.resample_shape_frequence(close_m15,cross_angle_h1)
            cross_angle_h4_ajusted  = us.resample_shape_frequence(close_m15,cross_angle_h4)
            cross_angle_d1_ajusted  = us.resample_shape_frequence(close_m15,cross_angle_d1)

            us.get_details_variable([cross_angle_m15_ajusted,cross_angle_m30_ajusted,cross_angle_h1_ajusted,cross_angle_h4_ajusted,cross_angle_d1_ajusted])
            
            print(cross_angle_d1_ajusted['trend'].iloc[2000])
            trades = Entry.run(
                close_m15,
                cross_angle_m15_ajusted,
                cross_angle_m30_ajusted,
                cross_angle_h1_ajusted,
                cross_angle_h4_ajusted,
                cross_angle_d1_ajusted,

            )
            long = trades.long.iloc[:, 0]
            short = trades.short.iloc[:, 0]

            exit = Exits.run(
                long,
                short,
                close_m15,
                high_m15,
                low_m15,
                r3_ajuster,
                s3_ajuster
            )

            us.print_trades(long,short,None,None,data)
            


            entries_long_df[ticker] = long.reset_index(drop=True)
            entries_short_df[ticker] = short.reset_index(drop=True)
            exits_long_df[ticker] = exit.long.reset_index(drop=True)
            exits_short_df[ticker] = exit.short.reset_index(drop=True)
            close_df[ticker] = data['close'].reset_index(drop=True)
            datetime_df[ticker] = datetime.reset_index(drop=True)

        size = np.array([60]*len(self.tickers))
        exits_long_df = exits_long_df.astype(np.float64).fillna(0)
        exits_short_df = exits_short_df.astype(np.float64).fillna(0)


        portfolio = vbt.Portfolio.from_order_func(
            close_df,  
            order_func_nb,  
            entries_long_df.values,  
            entries_short_df.values,  
            exits_long_df.values,
            exits_short_df.values,
            size,  
            freq=self.frequences,  
            init_cash=100000,  
            pre_sim_func_nb=pre_sim_func_nb,  
            post_order_func_nb=post_order_func_nb,  
            broadcast_named_args=dict(  
                long=entries_long_df.values,  
                short=entries_short_df.values,  
                exits_long_df=exits_long_df.values,  
                exits_short_df=exits_short_df.values,  
                size=size  
            )
        )
        us.rapport_backtest(portfolio,close_df,datetime_df,self.tickers)
        return portfolio

    def optimize(self,fitness_func,choice,period1,period2,params1,params2):
         
         if choice == 1:

            combinaison=[]
            for i in params1:
                for j in params2:
                    combinaison.append((i,j))
                    #Peut rajouter un troisieme param pour faire l'opti de 3 params (grand max)

            
            entries = Entry.run(param_product=True)


            long = entries.entries_long
            short = entries.entries_short

            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)
            close = close.reset_index(drop=True)
            close = pd.DataFrame(close)  
            close = close.iloc[:, :long.shape[1]]

            capital = 100000  
            size = np.array([(0.05 * capital) / (0.02 * close.iloc[0, i]) for i in range(close.shape[1])])
            size = np.array([size[0]] * long.shape[1])
            stop_loss = np.array([0.02] * long.shape[1])  
            take_profit = np.array([0.03] * long.shape[1])
            
            portfolio = vbt.Portfolio.from_order_func(
                close,  
                order_func_nb,
                long.values,  
                short.values,  
                stop_loss,  
                take_profit,  
                size,  
                freq=self.frequence_exec,
                init_cash=100000,
                pre_sim_func_nb=pre_sim_func_nb,
                post_order_func_nb=post_order_func_nb,
                broadcast_named_args=dict(
                    long=long.values,
                    short=short.values,
                    stop_loss=stop_loss,  
                    take_profit=take_profit,
                    size=size 
                )
            )

            perf = us.get_heatmap(portfolio,combinaison,fitness_func)
            param1,param2,metrics = us.get_best_param(perf)
            return param1,param2


    def wfa(self,period_OOS, period_IOS,params1):
 
        close_df = pd.DataFrame()

        for ticker in self.tickers:

            data = self.data[ticker]
            cycle = us.generate_wf_cycles(data, period_IOS, period_OOS)

            close = data['close']
            high = data['high']
            low = data['low']

            long_entries_index = []
            short_entries_index = []
            close_df[ticker] = close

            for i in range(len(cycle)):

                close_IOS = data["close"].iloc[cycle[i][0]:cycle[i][1]]
                close_OOS = data["close"].iloc[cycle[i][2]:cycle[i][3]]

                high_IOS = data["high"].iloc[cycle[i][0]:cycle[i][1]]
                high_OOS = data["high"].iloc[cycle[i][2]:cycle[i][3]]

                low_IOS = data["low"].iloc[cycle[i][0]:cycle[i][1]]
                low_OOS = data["low"].iloc[cycle[i][2]:cycle[i][3]]




                param1, param2 = self.optimize()

                #premiere itération on save la période IOS pour afficher l'entierté de mon jeu de donnée 
                if i == 0:
                    entries = Entry.run()
                    long_entries_index.extend(entries.entries_long[entries.entries_long].index)
                    short_entries_index.extend(entries.entries_short[entries.entries_short].index)


                entries = Entry.run()
                long_entries_index.extend(entries.entries_long[entries.entries_long].index)
                short_entries_index.extend(entries.entries_short[entries.entries_short].index)

            #Je copie close je remplace tout par False et je remplace par True les indices donnée par Entry
            entries_long = data["close"].copy()
            entries_long[:] = False 
            entries_long.loc[long_entries_index] = True  
            entries_short = data["close"].copy()
            entries_short[:] = False  
            entries_short.loc[short_entries_index] = True  
            #Mettre entries long et short dans le bon format 
            entries_long = entries_long.to_frame() 
            entries_short = entries_short.to_frame() 
            entries_long = entries_long.reset_index(drop=True) 
            entries_short = entries_short.reset_index(drop=True)  
            entries_long.columns = [ticker] 
            entries_short.columns = [ticker]
            entries_long = entries_long.astype(bool)  
            entries_short = entries_short.astype(bool)



            capital = 100000   
            size = np.array([(0.05 * capital) / (0.02 * close_df.iloc[0, i]) for i in range(len(self.tickers))])
            stop_loss = np.array([0.01] * len(self.tickers))  
            take_profit = np.array([0.02] * len(self.tickers))

        
            #Si ca fonctionne pas regarde bien que les shapes soit cohérente, le type de entries long et short ca doit etre des boolean

            portfolio = vbt.Portfolio.from_order_func(
                close_df,  
                order_func_nb,  
                entries_long.values,  
                entries_short.values,  
                stop_loss,  
                take_profit,  
                size,  
                freq=self.frequence_exec,  
                init_cash=100000,  
                pre_sim_func_nb=pre_sim_func_nb,  
                post_order_func_nb=post_order_func_nb,  
                broadcast_named_args=dict(  
                    long=entries_long.values,  
                    short=entries_short.values,  
                    stop_loss=stop_loss,  
                    take_profit=take_profit,  
                    size=size  
                )
            )
            us.get_pnl(portfolio)
            return portfolio



In [20]:
##UTILISATION DE LA STRAT##
tickers = ['USDJPY']
DATA = us.get_data_forex(tickers,['M15','M30','H1','H4','D1','W1'])


H:\Desktop\Environement_Trading_Developement\FunctionEssential\utils.py:259: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



In [ ]:
strat = Strategie(DATA,tickers,"15m")
strat.set_params_strategie('M15',21,2,7,5)
strat.backtest()

C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_5332\2472995118.py:31: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_5332\2472995118.py:31: RuntimeWarning:

divide by zero encountered in scalar divide



Variable : N/A
Type : <class 'pandas.core.frame.DataFrame'>
Shape : (562573, 3)


Variable : N/A
Type : <class 'pandas.core.frame.DataFrame'>
Shape : (562573, 3)


Variable : N/A
Type : <class 'pandas.core.frame.DataFrame'>
Shape : (562573, 3)


Variable : N/A
Type : <class 'pandas.core.frame.DataFrame'>
Shape : (562573, 3)


Variable : N/A
Type : <class 'pandas.core.frame.DataFrame'>
Shape : (562573, 3)


Nothing
